<a href="https://colab.research.google.com/github/sihan827/2021_Winter_PE/blob/main/train_process_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 훈련 과정 함수 정의
def train(model, train_loader, optimizer, epochs):
  # torch.nn.Module 클래스를 상속받은 모든 신경망 모델 클래스에는 train, eval 함수가 존재
  # train함수를 실행하면 모델 클래스의 training 변수가 True가 됨
  # 이를 통해 dropout이나 배치 정규화를 학습과정에서는 실행하고 테스트과정에서는 실행하지 않게 됨
  model.train()
  # 에폭은 전체 학습 데이터를 몇 번을 돌릴지의 횟수
  for epoch in range(epochs):
    # enumerate 함수를 통해 이터러블 객체인 train_loader에서 각 배치 데이터와 인덱스를 묶음
    # for문 내에서 train_loader의 배치의 인덱스, 데이터와 라벨을 순회함 
    for idx, (data, target) in enumerate(train_loader, 0):
      # 불러온 데이터 텐서와 라벨 텐서를 cuda/cpu에서 연산하도록 설정
      data, target = data.to(DEVICE), target.to(DEVICE)
      # optimizer를 정의할 시 인자로 모델의 파라미터(가중치, 편향)을 받음
      # zero_grad 함수를 호출하면 해당 파라미터들의 grad값이 0으로 초기화됨
      # 만약 각 step마다 0으로 초기화하지 않는다면 
      # 각 step마다 backward를 호출하여 계산된 파라미터들의 grad값이 계속 누적됨
      # 이는 손실값이 감소하는 방향으로 가지 않을 수 있음
      optimizer.zero_grad()
      # 존재하는 모델로 데이터에 대한 결과를 예측함
      output = model(data)
      # 원하는 방법으로 손실함수를 계산
      # 해당 경우에는 torch.nn.functional의 크로스 엔트로피 함수를 사용
      # torch의 cross_entropy 함수는 로그 소프트맥스 + NLLLoss의 기능을 겸함
      # 즉 모델 자체에서 소프트맥스를 따로 계산하지 않아도 인자로 전달하고 target으로 라벨을 전달하면
      # 알아서 모델의 결과에 로그 소프트맥스를 적용한 후 
      # target 라벨에 대하여 크로스 엔트로피 손실을 계산함
      # 즉 loss에는 모델의 파라미터(가중치, 편향 등)를 포함한 손실값 계산식이 저장됨
      loss = F.cross_entropy(output, target)
      # 계산을 원하는 식에 backward 함수 호출 시 해당 식에 대하여 back propagation 시행
      # (autograd가 True인)각 파라미터에 대한 grad 값이 계산됨
      loss.backward()
      # 선언한 optimizer의 step 함수 호출 시 
      # backward로 계산된 각 파라미터의 grad 값을 이용하여 
      # 선언한 optimizer에서 지정한 모델의 파라미터만 최적화시킴 
      optimizer.step()
      # 밑의 프린트문은 배치 60개마다 각 배치의 loss값의 평균을 출력하도록 한 것임
      # 이를 통해 현재 학습이 손실값이 감소하는 방향으로 진행중인지 판단
      # cross_entropy의 키워드 인자 중 reduction의 기본은 'mean'으로 
      # 이는 손실값을 어떻게 출력할 것인지를 정하는 인자로 기본값인 각 배치의 손실값 평균을 출력함
      # loss의 item 함수로 loss의 평균을 float형으로 출력
      if idx % 60 == 0:
        print('[%d %5d] loss : %f' % (epoch, idx, loss.item()))
  print('done!')